In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
from bs4 import BeautifulSoup

In [34]:
df = pd.read_json('db/news_vesti_db.json')

In [21]:
resp = req.get('https://www.vesti.ru/doc.html?id=2967578')
text = resp.text

In [29]:
def parse_vesti_news(text):
    soup = BeautifulSoup(text)
    news_text_list = soup.find_all(name="div", attrs={'class': "js-mediator-article"})
    
    if news_text_list:
        news_text = ''
        for news in news_text_list:
            news_text += news.get_text(strip=True)
    else:
        news_text = ''
    
    tags_elem = soup.find(name="meta", attrs={'name': "keywords"})
    tags = tags_elem.get('content') if tags_elem else ''
    
    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = description_elem.get('content') if description_elem else ''

    return news_text, tags, description

In [ ]:
for i, item in enumerate(df.iterrows()):

    if not df.loc[item[0], 'downloaded']:
        
        try:
            # Processing row from db
            url = item[1]['url']
            lastmod = item[1]['lastmod']
            csv_row = dict(zip(['news_id', 'url', 'lastmod'], [item[0], url, lastmod]))

            # Downloading file
            start = time.time()
            resp = req.get(url)
            if resp.status_code == 200:
                csv_row['news_text'], csv_row['tags'], csv_row['description'] = parse_vesti_news(resp.text)

            # Saving row to csv file
            with open('db/news_vesti.csv', 'a') as csv_file:
                writer = csv.DictWriter(csv_file, csv_row.keys())
                writer.writerow(csv_row)

            # News is written, logging it to DF
            df.loc[item[0], 'downloaded'] = True
            
            # Logging to console
            if i % 100 == 0 and i > 0:
                df.to_json('db/news_vesti_db.json')
                datetime = time.strftime('%Y-%m-%d %H:%M')
                print(datetime, ':', i, 'news downloaded,', int((df.shape[0] - i) * 1.1 / 60 / 60), 'hours remaining')

            # Calculating wait time (must be 1 sec minimum)
            end = time.time()
            wait = 1.1 - (end - start)
            time.sleep(wait)
            
        except:
            df.to_json('db/news_vesti_db.json')